In [1]:
import os
import sys
code_root = os.path.dirname(os.path.dirname(os.getcwd()))
if code_root not in sys.path:
    sys.path.append(code_root)
print(code_root)

/Users/matthewmcpartlon/VSCode/AttnPacker


In [2]:
import torch
from protein_learning.models.inference_utils import *


/Users/matthewmcpartlon/miniconda3/envs/py38/lib/python3.8/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [3]:
RESOURCE_ROOT = "/Users/matthewmcpartlon/Downloads/fbb_design_ft_inference/models"
#MODEL_NAME = "fbb_design_21_12_2022_16:00:06" #RX7-1
MODEL_NAME = "fbb_design_21_12_2022_15:57:43" # RX11
#MODEL_NAME = "fbb_design_21_12_2022_16:07:51" #RX7-2
runner = Inference(RESOURCE_ROOT, MODEL_NAME)
model = runner.get_model()
device = "cpu"#"cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()

[WARNING] : no value for key help, arg_group : optional arguments
[INFO] adding embeddings for res_ty: [1]
[INFO] adding embeddings for rel_pos: [1]
[INFO] adding embeddings for bb_dihedral: [1]
[INFO] adding embeddings for centrality: [0]
[INFO] adding embeddings for rel_sep: []
[INFO] adding embeddings for rel_dist: [0]
[INFO] adding embeddings for tr_ori: [1]
[INFO] adding embeddings for rel_chain: [1]
------------------------------------------------------
[INFO] Input feature embeddings

----[INFO] SCALAR (dim = 115)

--------res_ty
------------one_hot : 23
--------rel_pos
------------one_hot : 11
--------bb_dihedral
------------one_hot : 75
--------centrality
------------rbf : 6

----[INFO] PAIR (dim = 205)

--------rel_sep
--------rel_dist
------------rbf : 64
--------tr_ori
------------one_hot : 75
--------rel_chain
------------one_hot : 6
--------joint_pair_n_sep
------------rel_sep : 60
------------res_ty_a : 60
------------res_ty_b : 60

--------------------------------------

In [4]:
import time
pdb_path = "/Users/matthewmcpartlon/Downloads/casp_all/T0967.pdb"
example = runner.load_example(pdb_path = pdb_path)

In [5]:
start = time.time()
with torch.no_grad():
    model_out = model.forward(example.to(device), use_cycles = 1)
    print(f"Ran inference in {round(time.time() - start,3)} seconds")

/Users/matthewmcpartlon/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Ran inference in 1.382 seconds


In [6]:
#masked residue positions
seq_mask = default(
    example.input_features.seq_mask,torch.zeros(len(example.decoy)).bool()
)

# coords, pLDDT, and Sequence
res_feats = model_out.scalar_output
pred_coords = model_out.predicted_coords
pred_seq_labels = model_out.decoy_protein.seq_encoding
pred_seq_logits = None


# Predicted pLDDT
plddt_head = model.loss_fn.loss_fns["plddt"]
pred_plddt = plddt_head.get_expected_value(res_feats)
pred_plddt = pred_plddt

# Predicted Sequence
if "nsr" in model.loss_fn and torch.any(seq_mask):
    pred_seq_logits = model.loss_fn.loss_fns["nsr"].get_predicted_logits(res_feats)
    pred_seq_labels = torch.argmax(pred_seq_logits,dim=-1)

out = pred_coords, pred_seq_labels, pred_seq_logits, pred_plddt
fn = lambda x : x.detach().cpu().squeeze() if torch.is_tensor(x) else x
pred_coords, pred_seq_labels, pred_seq_logits, pred_plddt = map(fn,out)


In [7]:
pred_protein = make_predicted_protein(model_out, seq = pred_seq_labels)
pdb_out_path = f"./attn_packer/examples/{pred_protein.name}_packed.pdb"
print(f"saving pdb to {pdb_out_path}")
pred_protein.to_pdb(pdb_out_path)

saving pdb to ./attn_packer/examples/T0967_packed.pdb


In [8]:
from protein_learning.protein_utils.sidechains.project_sidechains import project_onto_rotamers

pp_pdb_out_path = f"./attn_packer/examples/{pred_protein.name}_packed_pp.pdb"

projected_coords, _ = project_onto_rotamers(
    atom_coords = pred_protein.atom_coords.unsqueeze(0),
    sequence = pred_protein.seq_encoding.unsqueeze(0),
    atom_mask = pred_protein.atom_masks.unsqueeze(0),
    steric_clash_weight=[0.5],
)

pred_protein.to_pdb(pp_pdb_out_path, coords=projected_coords.squeeze())


[fn: project_onto_rotamers] : Using device cpu
[INFO] Beginning rotamer projection
[INFO] Initial loss values
   [RMSD loss] = 0.474
   [Steric loss] = 8.396
   [Angle Dev. loss] = 0.0

beginning iter: 0, steric weight: 0.5
beginning iter: 1, steric weight: 0.5
[INFO] Final Loss Values
   [RMSD loss] = 0.479
   [Steric loss] = 4.348
   [Angle Dev. loss] = 0.001

